In [1]:
import pandas as pd
import numpy as np

#### READING DATA 

In [2]:
# Deferred data
def deffered_data():
    df_def_raw = pd.read_excel(r'D:\INFOSEC\New folder\deferred.xlsx',engine='openpyxl', sheet_name = 'Master file')  # 1
    df_def_1 = df_def_raw[['Tiger Email ID','Status']]
    df_def_1['Email Address [Required]'] = df_def_1['Tiger Email ID']
    df_def = df_def_1.dropna()
    df_def= df_def.rename(columns = {'Tiger Email ID	':'Email Address [Required]'})
    df_def['Email Address [Required]'] = df_def['Email Address [Required]'].apply(str.lower)
    return df_def

#### EMPLOYEE MASTER

In [3]:
def Employee_Master():
    #Infosec
    df_infosec = pd.read_csv(r'D:\INFOSEC\New folder\infosec.csv') # 2
    raw1=df_infosec[['Official Email Id','Date Of Exit']]
    raw1.columns = ['TA email','Termination date']
    #Contractors
    active_onshore = pd.read_excel(r'D:\INFOSEC\New folder\contractor.xlsx',engine='openpyxl', sheet_name = 'Active') # 3.1
    inactive_onshore = pd.read_excel(r'D:\INFOSEC\New folder\contractor.xlsx',engine='openpyxl', sheet_name = 'Terminated') # 3.2 Unused as email ID is missing
    # US_Employee-database-shared (Remove after 22nd May entries)
    active_US = pd.read_excel(r'D:\INFOSEC\New folder\us_database.xlsx',engine='openpyxl', sheet_name = 'US-CN') # 4.1
    inactive_US = pd.read_excel(r'D:\INFOSEC\New folder\us_database.xlsx',engine='openpyxl', sheet_name = 'Term') # 4.2
    raw2 = inactive_US[['TA email','Termination date']]
    raw3 =  raw1.append(raw2)
    raw3 = raw3.drop_duplicates(subset=['TA email'],keep='first')
    raw3.columns = ['Company Email ID','DOE']
    inactive_US = inactive_US.dropna(how='all')
    active_US = active_US.dropna(how='all')
    active_US_1 = active_US[['ID','Geo','Full Legal Name','TA Email','Date of Joining']]
    inactive_US_1 = inactive_US[['TA id','geo','Full Legal Name','TA email','Date of Joining']]
    cols = ['Employee_ID','Location','Name','Email','DOJ']
    active_US_1.columns = cols
    inactive_US_1.columns = cols
    active_US_1 = active_US_1.dropna(how='all')
    inactive_US_1 = inactive_US_1.dropna(how='all')
    inactive_US_1['Status'] = 'Inactive'
    active_US_1['Status'] = 'Active'
    world_total =  active_US_1.append(inactive_US_1)
    world_total = world_total.dropna()
    # world_total.loc[world_total.duplicated(subset = ['Employee_ID'],keep=False), :]
    world_total_final = world_total.drop_duplicates(subset=['Email'],keep='first') # True Active and Inactive Employees worldwide (except India)
    act_1 = active_onshore[['Contractor ID','GEO','Full Name','Official email ID','DOJ/Contract Start date']]
    act_1['Status'] = 'Active'
    act_1 = act_1.dropna()
    act_1.columns = cols + ['Status']
    infosec = df_infosec[['Employee Id','Current Country','Full Name','Official Email Id','Date Of Joining','Employment Status']]
    infosec.columns = cols + ['Status']
    # Assimilating Employee Master Data
    total = infosec.append([world_total_final,act_1])
    total = total.sort_values(by = ['Status'])
    total['Email'] = total['Email'].apply(str.lower)
    Emp_Master = total.drop_duplicates(subset=['Email'],keep='first')
    return [Emp_Master,raw3]

#### Gmail

In [4]:
def Gmail_data(Emp_Master,df_def):
    gmail = pd.read_csv(r'D:\INFOSEC\New folder\Gmail.csv') # 5
    gm_raw = gmail.copy()
    gmail = gmail[['First Name [Required]', 'Last Name [Required]',
        'Email Address [Required]','Org Unit Path [Required]','Status [READ ONLY]','Employee ID']]
    gmail['Email Address [Required]'] = gmail['Email Address [Required]'].apply(str.lower)
    # Emp_Master = Employee_Master()
    temp_gmail = Emp_Master[['Email','Status']]
    temp_gmail['Email_ID']= Emp_Master['Email']
    temp_gmail=temp_gmail.rename(columns = {'Email':'Email Address [Required]'})
    temp_gmail['Email Address [Required]'] = temp_gmail['Email Address [Required]'].apply(str.lower)
    gm = pd.merge(gmail,temp_gmail,on='Email Address [Required]',how='left')
    gm_final = pd.merge(gm,df_def,on='Email Address [Required]',how='left')
    gm_final.columns = ['First Name [Required]', 'Last Name [Required]',
        'Email Address [Required]', 'Org Unit Path [Required]',
        'Status [READ ONLY]', 'Employee ID', 'Status_Master', 'Employee Email',
        'Deferred', 'Def-Status']
    gm_final = gm_final[['First Name [Required]', 'Last Name [Required]',
        'Email Address [Required]', 'Org Unit Path [Required]',
        'Status [READ ONLY]', 'Employee ID','Employee Email','Status_Master',
        'Deferred','Def-Status']]
    id = gm_final[gm_final['Org Unit Path [Required]'].str.contains("Non-Employees")]
    id = id.rename(columns = {'Employee ID':'Employee_ID'})
    id = id[id['Employee_ID'].notna()]
    emp_temp = Emp_Master[['Employee_ID','Status']]
    emp_temp['Employee_ID'] = pd.to_numeric(emp_temp['Employee_ID'], errors='coerce')
    id2 = pd.merge(id,emp_temp,on='Employee_ID',how='left')
    gm_final = pd.merge(gm_final,id2[['Email Address [Required]','Status']],on='Email Address [Required]',how='left') 
    return [gm_final,gm_raw]

#### MDM

In [5]:
def MDM_data(Emp_Master,df_def):
    mdm = pd.read_csv(r'D:\INFOSEC\New folder\MDM.csv') # 6
    mdm = mdm[['Device Name', 'Name', 'Email','Status']]
    mdm['Email'] = mdm['Email'].apply(str.lower)
    mdm=mdm.rename(columns = {'Email':'Email Address [Required]'})
    temp_gmail = Emp_Master[['Email','Status']]
    temp_gmail['Email_ID']= Emp_Master['Email']
    temp_gmail=temp_gmail.rename(columns = {'Email':'Email Address [Required]'})
    mdm2 = pd.merge(mdm,temp_gmail,on='Email Address [Required]',how='left')
    mdm_final = pd.merge(mdm2,df_def,on='Email Address [Required]',how='left')
    mdm_final.columns = ['Device Name', 'Name', 'Email Address [Required]', 'Status','Employee Status',
        'Employee Email', 'Deferred', 'Def-Status']
    mdm_final = mdm_final[['Device Name', 'Name', 'Email Address [Required]', 'Status', 'Employee Email',
            'Employee Status', 'Deferred', 'Def-Status']]
    return mdm_final

#### AD

In [6]:
def AD_data(Emp_Master,df_def):
    ind_emp = pd.read_excel(r'D:\INFOSEC\New folder\india_employees.xlsx',engine='openpyxl') # 7
    ind_rest = pd.read_excel(r'D:\INFOSEC\New folder\india_restricted.xlsx',engine='openpyxl') # 8
    sgp = pd.read_excel(r'D:\INFOSEC\New folder\sgp.xlsx',engine='openpyxl') # 9
    uk = pd.read_excel(r'D:\INFOSEC\New folder\uk.xlsx',engine='openpyxl') # 10
    us = pd.read_excel(r'D:\INFOSEC\New folder\us.xlsx',engine='openpyxl') # 11
    cnd = pd.read_excel(r'D:\INFOSEC\New folder\canada.xlsx',engine='openpyxl') # 12
    # x1 = [ind_emp,ind_rest,sgp,uk,us]
    # x2 = ['IND','IND - Restricted','SGP','UK','US']
    x1 = [ind_emp,ind_rest,sgp,uk,us,cnd]
    x2 = ['IND','IND - Restricted','SGP','UK','US','CND']
    x3=pd.DataFrame()
    for i in range(0,len(x2)):
        x1[i]['Location'] = x2[i]
        # x1[i] = x1[i].drop(['Office'], axis=1)
        x3 = x3.append(x1[i])
    x3= x3.rename(columns = {'E-Mail Address':'Email Address [Required]'})
    x3['Email Address [Required]'] = x3['Email Address [Required]'].apply(str.lower)
    temp_gmail2 = Emp_Master[['Employee_ID','Email','Status']]
    temp_gmail2['Email_ID']= Emp_Master['Email']
    temp_gmail2= temp_gmail2.rename(columns = {'Email':'Email Address [Required]'})
    temp_gmail2['Email Address [Required]'] = temp_gmail2['Email Address [Required]'].apply(str.lower)
    temp_1 = pd.merge(x3,temp_gmail2,on='Email Address [Required]',how='left')
    ad = pd.merge(temp_1,df_def,on='Email Address [Required]',how='left')
    ad.columns = ['Name', 'Type', 'Description', 'Email Address [Required]', 'Location',
                    'Emp ID', 'Employee Status', 'Company Email Id', 'Deferred','Def-Status']
    return ad

#### Biometric


In [7]:
def bio_data(Emp_Master,df_def):
    bio = pd.read_excel(r"D:\INFOSEC\New folder\bio.xlsx",engine='openpyxl',index_col=False) # 13
    bio = bio[['EmployeeCode', 'EmployeeName', 'DeviceCode', 'Company', 'Department',
                        'Location', 'ShiftRoaster','Status', 'DOR']]
    new_emp = Emp_Master[Emp_Master['Location'].isin(['India'])]
    temp_gmail3 = new_emp[['Employee_ID','Email','Status']]
    temp_gmail3['EmployeeCode'] = temp_gmail3['Employee_ID']
    temp_gmail3['EmployeeCode'] = pd.to_numeric(temp_gmail3['EmployeeCode'], errors='coerce')
    temp_gmail3 = temp_gmail3[temp_gmail3['EmployeeCode'].notna()]
    bio_temp = pd.merge(bio,temp_gmail3,on='EmployeeCode',how='left')
    bio_temp= bio_temp.rename(columns = {'Email':'Email Address [Required]'})
    bio_final = pd.merge(bio_temp,df_def,on='Email Address [Required]',how='left')
    bio_final.columns = ['EmployeeCode', 'EmployeeName', 'DeviceCode', 'Company', 'Department',
                        'Location', 'ShiftRoaster', 'Status', 'DOR', 'Employee Id',
                        'Company Email Id', 'Employee Status', 'Deferred','Def-Status']
    return bio_final

#### Reconcilliation 

In [8]:
def recon_data(Emp_Master,gm_final,mdm_final,ad,bio_final,df_def):
    gm_recon = gm_final[['Email Address [Required]','Org Unit Path [Required]','Status [READ ONLY]']]
    gm_recon['Email'] = gm_recon['Email Address [Required]']
    gm_recon['Email'] = gm_recon['Email'].apply(str.lower)
    Emp_Master = Emp_Master.drop(['DOJ'], axis=1)
    recon = pd.merge(Emp_Master,gm_recon,on='Email',how='left')
    mdm_final_temp = mdm_final[['Email Address [Required]']]
    mdm_final_temp['Email'] = mdm_final_temp['Email Address [Required]']
    recon_2 = pd.merge(recon,mdm_final_temp,on='Email',how='left')
    recon_2 = recon_2.drop_duplicates(subset=['Email'],keep='first')
    recon_2 = pd.merge(recon,mdm_final_temp,on='Email',how='left')
    ad['Emp ID'] = pd.to_numeric(ad['Emp ID'], errors='coerce')
    ad = ad[ad['Emp ID'].notna()]
    ad_temp = ad[['Email Address [Required]']]
    ad_temp['Email'] = ad_temp['Email Address [Required]']
    recon_3 = pd.merge(recon_2,ad_temp,on='Email',how='left')
    recon_3['Employee_ID'] = pd.to_numeric(recon_3['Employee_ID'], errors='coerce')
    ad_temp2 = ad[['Emp ID']]
    ad_temp2['Employee_ID'] = ad_temp2['Emp ID']
    recon_4 = pd.merge(recon_3,ad_temp2,on='Employee_ID',how='left')
    recon_4 = recon_4.drop_duplicates(subset=['Email'],keep='first')
    bio_tempp = bio_final[['EmployeeCode','Status']]
    bio_tempp['Employee_ID'] = bio_tempp['EmployeeCode']
    recon_5 = pd.merge(recon_4,bio_tempp,on='Employee_ID',how='left')
    recon_5 = recon_5.drop_duplicates(subset=['Email'],keep='first')
    df_def = df_def.rename(columns = {'Email Address [Required]':'Email'})
    recon_6 = pd.merge(recon_5,df_def,on='Email',how='left')
    recon_6.columns = ['Employee ID', 'Location', 'Full Name', 'Company Email ID', 'Employee Status',
        'Gmail', 'OU','Status [READ ONLY]', 'MDM','AD Email', 'AD EmpID', 
        'Biometric EID', 'Biometric Status','Deffered', 'Def-Status']
    recon_6 = recon_6[['Employee ID', 'Location', 'Full Name', 'Company Email ID', 'Employee Status',
                'Deffered', 'Def-Status', 'Gmail', 'OU','Status [READ ONLY]', 'MDM',
                'AD Email', 'AD EmpID','Biometric EID', 'Biometric Status']]
    return recon_6

#### Analysis

In [9]:
def Active_Inactive_Analysis(Emp_Master,recon,raw3):
    emp_recon_temp = Emp_Master[['Email','DOJ']]
    emp_recon_temp.columns = ['Company Email ID','DOJ']
    recon_doj = pd.merge(recon,emp_recon_temp,on='Company Email ID',how='left')

    # RECON ACTIVE & INACTIVE
    recon_doj['active_gmail_flag'] = np.where(((recon_doj["Employee Status"]=='Active') & 
                                            (recon_doj['Gmail'].isna())),1,0)
    recon_doj_active_gmail = recon_doj[recon_doj['active_gmail_flag']==1]
    recon_doj_active_gmail = pd.merge(recon_doj_active_gmail,raw3,on='Company Email ID',how='left')

    

    recon_doj['active_AD_flag'] = np.where(((recon_doj["Employee Status"]=='Active') & 
                                            (recon_doj['AD Email'].isna()) & 
                                            (recon_doj['AD EmpID'].isna()) & 
                                            (recon_doj["Location"]=='India') ),1,0)
    recon_doj_active_AD = recon_doj[recon_doj['active_AD_flag']==1]
    recon_doj_active_AD = pd.merge(recon_doj_active_AD,raw3,on='Company Email ID',how='left')


    recon_doj['Inactive_gmail_flag'] = np.where(((recon_doj["Employee Status"]=='Inactive') & 
                                                (recon_doj['Def-Status']!='Awaiting Joining') & 
                                                (~recon_doj['Gmail'].isna()) ),1,0)
    recon_doj_inactive_gmail = recon_doj[recon_doj['Inactive_gmail_flag']==1]
    recon_doj_inactive_gmail = pd.merge(recon_doj_inactive_gmail,raw3,on='Company Email ID',how='left')    

    recon_doj['Inactive_mdm_flag'] = np.where(((recon_doj["Employee Status"]=='Inactive') & 
                                            (recon_doj['Def-Status']!='Awaiting Joining') & 
                                                (~recon_doj['MDM'].isna()) ),1,0)
    recon_doj_inactive_mdm = recon_doj[recon_doj['Inactive_mdm_flag']==1]
    recon_doj_inactive_mdm = pd.merge(recon_doj_inactive_mdm,raw3,on='Company Email ID',how='left')    

    recon_doj['Inactive_AD_flag'] = np.where(((recon_doj["Employee Status"]=='Inactive') & 
                                            (recon_doj['Def-Status']!='Awaiting Joining') & 
                                            (~recon_doj['AD Email'].isna()) ),1,0)
    recon_doj_inactive_AD = recon_doj[recon_doj['Inactive_AD_flag']==1]
    recon_doj_inactive_AD = pd.merge(recon_doj_inactive_AD,raw3,on='Company Email ID',how='left')    

    recon_doj['Inactive_bio_flag'] = np.where(((recon_doj["Employee Status"]=='Inactive') & 
                                            (recon_doj['Def-Status']!='Awaiting Joining') & 
                                            (recon_doj["Biometric Status"]=='Working')),1,0)
    recon_doj_inactive_bio = recon_doj[recon_doj['Inactive_bio_flag']==1]
    recon_doj_inactive_bio = pd.merge(recon_doj_inactive_bio,raw3,on='Company Email ID',how='left')    

    return [recon_doj_active_gmail,recon_doj_active_AD,recon_doj_inactive_gmail,recon_doj_inactive_mdm,recon_doj_inactive_AD,recon_doj_inactive_bio]

In [10]:
#MDM
def mdm_analysis(mdm,Emp_Master):
    mdm['Inactive_flag'] = np.where(((mdm["Employee Status"]!='Active') & 
                                    (mdm['Def-Status']!='Awaiting Joining')),1,0)
    analysis_mdm = mdm[mdm['Inactive_flag']==1][['Email Address [Required]']]
    analysis_mdm.columns = ['Email']
    analysis_mdm = pd.merge(analysis_mdm,Emp_Master[['Email','DOJ']],on='Email',how='left')
    return analysis_mdm

In [11]:
#AD 
def ad_analysis(ad,decide,Emp_Master):
    if decide == 'yes':
        emp_temp = Emp_Master[['Employee_ID','Status']]
        emp_temp['Employee_ID'] = pd.to_numeric(emp_temp['Employee_ID'], errors='coerce')
        emp_temp = emp_temp.rename(columns = {'Employee_ID':'Emp ID'})
        emp_temp = emp_temp[emp_temp['Emp ID'].notna()]
        ad['Emp ID'] = pd.to_numeric(ad['Emp ID'], errors='coerce')
        ad = ad[ad['Emp ID'].notna()]
        ad = pd.merge(ad,emp_temp,on='Emp ID',how='left')  

        ad['Inactive_flag'] = np.where(((ad["Employee Status"]!='Active') & 
                                        (ad['Def-Status']!='Awaiting Joining') &
                                        (ad['Status']!='Active')),1,0)
        analysis_ad = ad[ad['Inactive_flag']==1]
    elif decide =='no':
        ad['Inactive_flag'] = np.where(((ad["Employee Status"]!='Active') & 
                                        (ad['Def-Status']!='Awaiting Joining')),1,0)
        analysis_ad = ad[ad['Inactive_flag']==1]
    return analysis_ad

In [12]:
#BIOMETRIC
def bio_analysis(bio_final,Emp_Master):
    bio_final['Inactive_flag'] = np.where(((bio_final["Employee Status"]!='Active') & 
                                    (bio_final['Def-Status']!='Awaiting Joining')&
                                    (bio_final['Status']=='Working')),1,0)
    analysis_bio = bio_final[bio_final['Inactive_flag']==1]
    analysis_bio = analysis_bio[~analysis_bio['EmployeeName'].str.contains("isitor", na=False)]
    analysis_bio = analysis_bio[~analysis_bio['EmployeeName'].str.contains("Facility", na=False)]
    analysis_bio = analysis_bio[~analysis_bio['EmployeeName'].str.contains("SECURITY", na=False)]
    analysis_bio = analysis_bio[~analysis_bio.EmployeeName.str.contains(r'\d', na=False)]
    analysis_bio = analysis_bio.rename(columns = {'EmployeeName':'Name'})
    analysis_bio = pd.merge(analysis_bio,Emp_Master[['Name','DOJ']],on='Name',how='left')
    return analysis_bio


In [13]:
#Gmail
def gmail_analysis(gmail,gm_raw):
    analysis_gmail = gmail[gmail['Org Unit Path [Required]'].str.contains("Non")]
    analysis_gmail['Non_flag'] = np.where((analysis_gmail['Employee ID'].isna()),1,0)
    analysis_gmail_1 = analysis_gmail[analysis_gmail['Non_flag']==1]
    analysis_gmail_1 = pd.merge(analysis_gmail_1[['Email Address [Required]', 'Org Unit Path [Required]',
        'Status [READ ONLY]', 'Employee ID']],gm_raw[['Email Address [Required]','Last Sign In [READ ONLY]']],on='Email Address [Required]',how='left')
    gmail['Inactive_flag'] = np.where(((gmail["Status_Master"]!='Active')& 
                                            (gmail['Def-Status']!='Awaiting Joining')& 
                                            (gmail['Status']!='Active')),1,0)
    analysis_gmail_2 = gmail[gmail['Inactive_flag']==1]
    analysis_gmail_2 = pd.merge(analysis_gmail_2[['Email Address [Required]', 'Org Unit Path [Required]',
        'Status [READ ONLY]', 'Employee ID']],gm_raw[['Email Address [Required]','Last Sign In [READ ONLY]']],on='Email Address [Required]',how='left')
    return [analysis_gmail_1,analysis_gmail_2]

In [22]:
bio_final['Inactive_flag'] = np.where(((bio_final["Employee Status"]!='Active') & 
                                (bio_final['Def-Status']!='Awaiting Joining')&
                                (bio_final['Status']=='Working')),1,0)
analysis_bio = bio_final[bio_final['Inactive_flag']==1]
analysis_bio = analysis_bio[~analysis_bio['EmployeeName'].str.contains("isitor", na=False)]
analysis_bio = analysis_bio[~analysis_bio['EmployeeName'].str.contains("Facility", na=False)]
analysis_bio = analysis_bio[~analysis_bio['EmployeeName'].str.contains("SECURITY", na=False)]
analysis_bio = analysis_bio[~analysis_bio.EmployeeName.str.contains(r'\d', na=False)]
analysis_bio = analysis_bio.rename(columns = {'EmployeeName':'Name'})
analysis_bio = pd.merge(analysis_bio,Emp_Master[['Name','DOJ']],on='Name',how='left')

#### Processing files & Saving Files


In [14]:
def saving(decide):
    df_def = deffered_data()
    master_raw = Employee_Master()
    Emp_Master = master_raw[0]
    raw3 = master_raw[1]
    gm_final = Gmail_data(Emp_Master,df_def)[0]
    gm_raw = Gmail_data(Emp_Master,df_def)[1]
    mdm = MDM_data(Emp_Master,df_def)
    ad = AD_data(Emp_Master,df_def)
    bio_final = bio_data(Emp_Master,df_def)
    recon = recon_data(Emp_Master,gm_final,mdm,ad,bio_final,df_def)
    dfs = Active_Inactive_Analysis(Emp_Master,recon,raw3)
    recon_doj_active_gmail = dfs[0]
    recon_doj_active_AD = dfs[1]
    recon_doj_inactive_gmail = dfs[2]
    recon_doj_inactive_mdm = dfs[3]
    recon_doj_inactive_AD = dfs[4]
    recon_doj_inactive_bio = dfs[5]
    analysis_mdm = mdm_analysis(mdm,Emp_Master)
    analysis_ad = ad_analysis(ad,decide,Emp_Master)
    analysis_bio = bio_analysis(bio_final,Emp_Master)
    analysis_gmail_1 =gmail_analysis(gm_final,gm_raw)[0]
    analysis_gmail_2 =gmail_analysis(gm_final,gm_raw)[1]

    df = pd.DataFrame([[np.NaN]],columns = ['Active - Gmail'])
    df2 = pd.DataFrame([[np.NaN]],columns = ['Active - AD'])
    df3 = pd.DataFrame([[np.NaN]],columns = ['Inactive - Gmail'])
    df4 = pd.DataFrame([[np.NaN]],columns = ['Inactive - MDM'])
    df5 = pd.DataFrame([[np.NaN]],columns = ['Inactive - AD'])
    df6 = pd.DataFrame([[np.NaN]],columns = ['Inactive - Biometric'])
    df7 = pd.DataFrame([[np.NaN]],columns = ['Analysis - Gmail - NON Employees'])
    df8 = pd.DataFrame([[np.NaN]],columns = ['Analysis - Gmail - InActive'])
    with pd.ExcelWriter("6way_Final.xlsx") as writer:

        recon.to_excel(writer, sheet_name="Reconcilliation", index=False)
        Emp_Master.to_excel(writer, sheet_name="Emp_Master", index=False)
        gm_final.to_excel(writer, sheet_name="Gmail", index=False)
        mdm.to_excel(writer, sheet_name="MDM", index=False)
        ad.to_excel(writer, sheet_name="AD", index=False)
        bio_final.to_excel(writer, sheet_name="Biometric", index=False)

        df.to_excel(writer,sheet_name='Analysis A+I',index= False) 
        recon_doj_active_gmail[['Full Name','Company Email ID','DOJ','DOE','Location']].to_excel(writer,sheet_name='Analysis A+I',
                    startrow=1 , index= False) 
        
        df2.to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + 3 , startcol=0,index= False) 
        recon_doj_active_AD[['Full Name','Company Email ID','DOJ','DOE']].to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + 4 , startcol=0,index= False) 
        
        df3.to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] + 6 , startcol=0,index= False) 
        recon_doj_inactive_gmail[['Full Name','Company Email ID','DOJ','DOE','Location']].to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] + 7  , startcol=0,index= False) 
        
        df4.to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] +recon_doj_inactive_gmail.shape[0]+ 9 , startcol=0,index= False) 
        recon_doj_inactive_mdm[['Full Name','Company Email ID','DOJ','DOE','Location']].to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] +recon_doj_inactive_gmail.shape[0]+ 10  , startcol=0,index= False)
    
        df5.to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] +recon_doj_inactive_gmail.shape[0]+recon_doj_inactive_mdm.shape[0] + 12 , startcol=0,index= False) 
        recon_doj_inactive_AD[['Full Name','Company Email ID','DOJ','DOE','Location']].to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] +recon_doj_inactive_gmail.shape[0]+recon_doj_inactive_mdm.shape[0] + 13  , startcol=0,index= False)
        
        df6.to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] +recon_doj_inactive_gmail.shape[0]+recon_doj_inactive_mdm.shape[0] + recon_doj_inactive_AD.shape[0] + 15 , startcol=0,index= False) 
        recon_doj_inactive_bio[['Full Name','Company Email ID','DOJ','DOE','Location']].to_excel(writer,sheet_name='Analysis A+I',
                    startrow=recon_doj_active_gmail.shape[0] + recon_doj_active_AD.shape[0] +recon_doj_inactive_gmail.shape[0]+recon_doj_inactive_mdm.shape[0] + recon_doj_inactive_AD.shape[0] + 16  , startcol=0,index= False)

        df4.to_excel(writer,sheet_name='Raw Analysis',index= False) 
        analysis_mdm.to_excel(writer,sheet_name='Raw Analysis',
                    startrow=1  , startcol=0,index= False)
    
        df5.to_excel(writer,sheet_name='Raw Analysis',
                    startrow=analysis_mdm.shape[0] + 3 , startcol=0,index= False) 
        analysis_ad.to_excel(writer,sheet_name='Raw Analysis',
                    startrow=analysis_mdm.shape[0] + 4  , startcol=0,index= False)
        
        df6.to_excel(writer,sheet_name='Raw Analysis',
                    startrow=analysis_mdm.shape[0] +analysis_ad.shape[0]  + 6 , startcol=0,index= False) 
        analysis_bio[['EmployeeCode', 'Name','DOJ']].to_excel(writer,sheet_name='Raw Analysis',
                    startrow=analysis_mdm.shape[0] +analysis_ad.shape[0]  + 7  , startcol=0,index= False)

        df7.to_excel(writer,sheet_name='Gmail Analysis',index= False) 
        analysis_gmail_1.to_excel(writer,sheet_name='Gmail Analysis',
                    startrow=1  , startcol=0,index= False)
    
        df8.to_excel(writer,sheet_name='Gmail Analysis',
                    startrow=analysis_gmail_1.shape[0] + 3 , startcol=0,index= False) 
        analysis_gmail_2.to_excel(writer,sheet_name='Gmail Analysis',
                    startrow=analysis_gmail_1.shape[0] + 4  , startcol=0,index= False)

In [15]:
# yes or no for vlookup on emp id on AD sheet analysis 
decide ='yes'

In [ ]:
saving(decide)